In [5]:

from scipy.spatial.distance import jensenshannon
import numpy as np
import sys

sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model, sample_gapsplit, plot_grid_95_reactions
from load_modify_sample_utils import sample_dingo, sample_optgp, sampling_statistics
from correlations_utils import compute_copula
from correlations_utils import plot_copula


In [6]:

ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

In [7]:

ec_cobra_model_condition_1, ec_dingo_model_condition_1 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=100)
ec_cobra_model_condition_2, ec_dingo_model_condition_2 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=50)


samples_dingo_condition_1 = sample_dingo(ec_dingo_model_condition_1, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_1.shape)

samples_dingo_condition_2 = sample_dingo(ec_dingo_model_condition_2, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_2.shape)


Read LP format model from file /tmp/tmpoetq4c10.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmp7xflxwhi.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
phase 1: number of correlated samples = 500, effective sample size = 8, ratio of the maximum singilar value over the minimum singular value = 1086.65
phase 2: number of correlated samples = 500, effective sample size = 7, ratio of the maximum singilar value over the minimum singular value = 277.058
phase 3: number of correlated samples = 500, effective sample size = 6, ratio of the maximum singilar value over the minimum singular value = 96.1823
phase 4: number of correlated samples = 500, effective sample size = 38, ratio of the maximum singilar value over the minimum singular value = 91.1664
phase 5: number of correlated sampl

[5]maximum marginal PSRF: 1.12213



[5]total ess 2040: number of correlated samples = 6700


(95, 6700)
phase 1: number of correlated samples = 500, effective sample size = 3, ratio of the maximum singilar value over the minimum singular value = 1533.17
phase 2: number of correlated samples = 500, effective sample size = 69, ratio of the maximum singilar value over the minimum singular value = 2.0516
phase 3: number of correlated samples = 2400, effective sample size = 857
phase 4: number of correlated samples = 2400, effective sample size = 860
phase 5: number of correlated samples = 900, effective sample size = 283
[5]total ess 2072: number of correlated samples = 6700


(95, 6700)


[5]maximum marginal PSRF: 1.00744


In [ ]:


def copula_tail_dependence(copula, cop_coeff_1, cop_coeff_2, cop_coeff_3, indicator_cutoff):

    rows, cols = copula.shape
    
    red_mass = 0
    blue_mass = 0
        
    top_left_count = 0
    bottom_right_count = 0
    top_right_count = 0
    bottom_left_count = 0
    
    top_left = []
    bottom_right = []
    top_right = []
    bottom_left = []
    
                    
    for row in range(rows):
        for col in range(cols):
            # values in the diagonal
            if ((row-col >= -cop_coeff_1*rows) & (row-col <= cop_coeff_1*rows)): 
                # values near the top left
                if (row+col < cop_coeff_2*rows -1):
                    red_mass = red_mass + copula[row][col]       
                    top_left_count = top_left_count + 1
                    top_left.append((row, col))
                    
                # values near the bottom right
                elif (row+col > cop_coeff_3*rows -1):
                    red_mass = red_mass + copula[row][col]       
                    bottom_right_count = bottom_right_count + 1
                    bottom_right.append((row, col))
            
            # values in the other diagonal
            else:
                # values near the top right and bottom left corner
                if (row+col >= cop_coeff_2*rows -1) & (row+col <= cop_coeff_3*rows -1):                    
                    # values near the top right
                    if row <= rows / 2:
                        blue_mass = blue_mass + copula[row][col]       
                        top_right_count = top_right_count + 1
                        top_right.append((row, col))
                    
                    # values near the bottom left
                    elif row > rows / 2:
                        blue_mass = blue_mass + copula[row][col]       
                        bottom_left_count = bottom_left_count + 1
                        bottom_left.append((row, col))
                                                

    print(top_left, "\n", bottom_right, "\n", top_right, "\n" , bottom_left)
    print(top_left_count, bottom_right_count, top_right_count, bottom_left_count)
    print(red_mass/blue_mass)

    
    
    

n = 5
copula = np.zeros((n, n))

cop_coeff = 0.2
cop_coeff_1 = cop_coeff
cop_coeff_2 = 1 - cop_coeff
cop_coeff_3 = 1 + cop_coeff

n1 = ec_dingo_reactions.index("PFK")
#n1 = ec_dingo_reactions.index("PGL")
n2 = ec_dingo_reactions.index("PYK")
flux1 = samples_dingo_condition_1[n1]
flux2 = samples_dingo_condition_1[n2]
data_flux1=[flux1,ec_dingo_reactions[n1]]
data_flux2=[flux2,ec_dingo_reactions[n2]]



plot_copula(data_flux1, data_flux2, n = n)
n1 = ec_dingo_reactions.index("PFK")

copula = compute_copula(flux1, flux2, n = n)
copula_tail_dependence(copula, cop_coeff_1, cop_coeff_2, cop_coeff_3, indicator_cutoff=1)




[(0, 0), (0, 1), (1, 0), (1, 1)] 
 [(3, 3), (3, 4), (4, 3), (4, 4)] 
 [(0, 3), (0, 4), (1, 3), (1, 4)] 
 [(3, 0), (3, 1), (4, 0), (4, 1)]
4 4 4 4
0.693061544492356


In [14]:

computed_copula = copula
computed_copula_flat = computed_copula.flatten()


reference_copula = np.full( (n, n), (1 / (n*n)) )
reference_copula_flat = reference_copula.flatten()


dist = jensenshannon(reference_copula_flat, computed_copula_flat)
print(dist)


0.11538769907325248


In [45]:


cells = 5
reference_copula = np.full( (cells, cells), (1 / (cells*cells)) )
reference_copula_1_flat = reference_copula.flatten()


reference_copula_2 = np.array([[0.25, 0.25, 0, 0, 0],
                               [0.25, 0.25, 0, 0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0, 0,  0]])

reference_copula_2_flat = reference_copula_2.flatten()


reference_copula_3 = np.array([[1, 0, 0, 0, 0],
                               [0, 0, 0, 0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0,  0, 0],
                               [0, 0, 0, 0,  0]])

reference_copula_3_flat = reference_copula_3.flatten()


# Compute Jensen-Shannon distance (range: 0 to 1)
dist = jensenshannon(reference_copula_1_flat, reference_copula_1_flat)
print(dist)

dist = jensenshannon(reference_copula_1_flat, reference_copula_2_flat)
print(dist)

dist = jensenshannon(reference_copula_1_flat, reference_copula_3_flat)
print(dist)



0.0
0.6785698641197397
0.7799839057460577
